In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/'
os.environ['CONDA_PREFIX'] = ''

import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
import wandb
import os
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from sklearn.dummy import DummyClassifier
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PrefixTuningConfig, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from peft import PeftModel, PeftConfig
from glob import glob 
from collections import defaultdict, Counter
from os.path import basename


import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # replace the 0 with other gpu ids
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = "cuda:0"
torch.cuda.set_device(0)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /opt/anaconda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda115.so...


/opt/anaconda/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('lib')}
  warn(msg)
/opt/anaconda/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning:  did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_name_or_path = "google/flan-t5-xl"
tokenizer_name_or_path = "google/flan-t5-xl"
best_model_dir = '/shared/2/projects/contextual-appropriateness/models/peft/google/flan-t5-xl/seed-12345//best'

text_column = "text"
label_column = "answer"
max_length = 192
lr = 1e-2
num_epochs = 20
batch_size = 128

In [15]:
only_pride_verbalizaion=  {
    'parent': 'a parent to their child',
    'child': 'a child to their parent',
    'sibling': 'a person to their sibling',
    'married': 'a person to their spouse', 
    'friend': 'a person to their friend',
    'enemy': 'a person to their enemy',
    'lover':'a person to their lover',
    'colleague': 'a person to their colleague',
    'doctor': 'a person to a medical professional',
    'commercial':'a person to someone in a commercial relationship',
    'direct_report': 'a person to their boss',
    'boss': 'a boss to their employee',
    "student":'a student to their teacher',
    "teacher": 'a teacher to their student',
    "engaged": 'a person to their fiancee',
    "classmate": 'a person to their classmate',
    "acquaintance": 'a person to someone distant',
    'church':'a person to someone in their church'
}
v2r = {v:r for r,v in only_pride_verbalizaion.items()}

In [16]:
def generate_text(row):
    desc = row['relationship']
    text = 'Rate whether it is inappropriate for this message to be said in the following social setting?\nsetting: from %s\nmessage: %s\nanswer (yes or no):' % (desc, row['quote'])
    return text

In [17]:
PRIDE_PATH = '/shared/2/projects/contextual-appropriateness/data/PRIDE/data/'
pride_df = pd.read_csv(PRIDE_PATH + '/cleaned_pride_data.csv')

pride_df['verbalization'] = pride_df.relationship
pride_df['relationship'] = pride_df.verbalization.map(v2r)

pride_df.head()

,quote,relationship,inst_id,length,text,answer,verbalization
0,"Sir , that is true .",enemy,0,4,Rate whether it is inappropriate for this mess...,yes,a person to their enemy
1,"Yes , sir .",enemy,1,2,Rate whether it is inappropriate for this mess...,yes,a person to their enemy
2,"Well , I am sure we are all glad to see you ca...",enemy,2,16,Rate whether it is inappropriate for this mess...,yes,a person to their enemy
3,And what is the state of your memory now ?,enemy,4,9,Rate whether it is inappropriate for this mess...,yes,a person to their enemy
4,"I am sorry , what was the question ?",enemy,5,7,Rate whether it is inappropriate for this mess...,yes,a person to their enemy


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

# Generate Dataframe for all the PRIDE relationships

In [13]:
rels = list(set(pride_df.relationship))

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-13-22f3538e63e7>:1 in <module>                                             │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pride_df' is not defined

In [13]:
def verbalize(desc, quote):
    text = 'Rate whether it is inappropriate for this message to be said in the following social setting?\nsetting: from %s\nmessage: %s\nanswer (yes or no):' % (desc, quote)
    return text

In [15]:
all_rel_pride_df = defaultdict(list)
for row in pride_df.itertuples():
    quote = row.quote
    for rel in rels:
        all_rel_pride_df['pride_id'].append(row.inst_id)
        desc = only_pride_verbalizaion[rel]
        
        all_rel_pride_df['text'].append(verbalize(desc, quote))
        all_rel_pride_df['relationship'].append(rel)
        
all_rel_pride_df = pd.DataFrame(all_rel_pride_df)
all_rel_pride_df['answer'] = 'yes'
all_rel_pride_df.head()

,pride_id,text,relationship,answer
0,0,Rate whether it is inappropriate for this mess...,boss,yes
1,0,Rate whether it is inappropriate for this mess...,married,yes
2,0,Rate whether it is inappropriate for this mess...,church,yes
3,0,Rate whether it is inappropriate for this mess...,child,yes
4,0,Rate whether it is inappropriate for this mess...,doctor,yes


In [16]:
len(all_rel_pride_df)

860418

In [17]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_pride_df.to_csv(peft_outdir + 'all_rel_PRIDE.pride-rels.csv', index=False)

In [4]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
all_rel_pride_df = pd.read_csv(peft_outdir + 'all_rel_PRIDE.pride-rels.csv')

In [6]:
all_rel_pride_ds = Dataset.from_pandas(all_rel_pride_df)
all_rel_pride_dd = DatasetDict()
all_rel_pride_dd['eval'] = all_rel_pride_ds

In [7]:
processed_all_rel_pride = all_rel_pride_dd.map(
    preprocess_function,
    batched=True,
    num_proc=8,
    remove_columns=all_rel_pride_dd["eval"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset (num_proc=8):   0%|          | 0/430209 [00:00<?, ? examples/s]

In [8]:
config = PeftConfig.from_pretrained(best_model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, best_model_dir)
model.to(device)
model.eval()
print('model loaded')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded


In [9]:
all_rel_pride_dataloader = DataLoader(
    processed_all_rel_pride['eval'], shuffle=False, collate_fn=default_data_collator, 
    batch_size=154, pin_memory=True
)

all_rel_pride_preds = []
for step, batch in enumerate(tqdm(all_rel_pride_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    all_rel_pride_preds.extend(
        tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), 
                               skip_special_tokens=True)
    )

  0%|          | 0/2794 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
all_rel_pride_bin_preds = [1 if p == 'yes' else 0 for p in all_rel_pride_preds]

In [10]:
all_rel_pride_df['prediction'] = all_rel_pride_preds

<ipython-input-10-c8c35db05079>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part1['prediction'] = all_rel_pride_preds


In [11]:
Counter(all_rel_pride_preds)

Counter({'yes': 205866, 'no': 224343})

In [12]:
all_rel_pride_df.head()

,pride_id,text,relationship,answer,prediction
0,0,Rate whether it is inappropriate for this mess...,boss,yes,yes
1,0,Rate whether it is inappropriate for this mess...,married,yes,yes
2,0,Rate whether it is inappropriate for this mess...,church,yes,yes
3,0,Rate whether it is inappropriate for this mess...,child,yes,yes
4,0,Rate whether it is inappropriate for this mess...,doctor,yes,yes


In [13]:
peft_outdir = '/shared/2/projects/contextual-appropriateness/results/peft/'
part1.to_csv(peft_outdir + 'all_rel_PRIDE.pride-rels.labeled.csv', index=False)

In [36]:
Counter(part1.relationship)

Counter({'patient': 23901,
         'neighbor': 23901,
         'domestic_partner': 23901,
         'sibling': 23901,
         'boss': 23901,
         'employee_in_large_company': 23901,
         'married': 23901,
         'teacher': 23901,
         'law_enforcement': 23901,
         'client': 23901,
         'acquaintance': 23901,
         'engaged': 23901,
         'dating': 23901,
         'adopted_child': 23901,
         'enemy': 23901,
         'classmate': 23901,
         'old_friend': 23901,
         'club_member': 23901,
         'mentor': 23901,
         'colleague': 23901,
         'uncle_aunt': 23901,
         'rival': 23901,
         'person_with_authority': 23901,
         'direct_report': 23901,
         'complete_stranger': 23901,
         'child': 23900,
         'landlord': 23900,
         'competitor': 23900,
         'grandchild': 23900,
         'childhood_friend': 23900,
         'best_friend': 23900,
         'ex_dating': 23900,
         'student': 23900,
        